In [1]:
#inspiration https://medium.com/lets-talk-ml/guide-to-your-very-own-a-i-virtual-assistant-in-python-d8120ff228a8

#!pip install pyttsx3
#!pip install speechRecognition
#!pip install pyaudio
#!pip install wikipedia
#pip install googletrans  virkede ikke
#pip install googletrans==4.0.0-rc1  # new update der virker
# #not working !pip install ecapture



In [1]:

import speech_recognition as sr
import pyttsx3
from gtts import gTTS
import playsound

import datetime
import wikipedia
import webbrowser
import os
import time
import subprocess
#from ecapture import ecapture as ec
import wolframalpha
import json
import requests
import googletrans


In [2]:
#init webbrowser
webbrowser.register('chrome',
	None,
	webbrowser.BackgroundBrowser("C://Program Files (x86)//Google//Chrome//Application//chrome.exe"))
#webbrowser.get('chrome').open(url)


#init takeCommand
#https://cloud.google.com/speech-to-text/docs/languages
lang_dk = 'da-DK'
lang_no = 'no-NO'
lang_eng_GB = 'en-GB'
lang_eng_US = 'en-US'
lang_viet = 'vn-VN'



In [3]:
#VOICE 
#import pyttsx3

def set_voice(gender=0, rate=125,volume=.7):
    engine = pyttsx3.init('sapi5')

    #voice setttings
    #engine.setProperty('voice', voices[gender].id) #male = 0, female = 1
    engine.setProperty('volume',volume)    # setting up volume level  between 0 and 1
    engine.setProperty('rate', rate)     # setting up new voice rate
    return engine
    #engine = set_voice(gender=0, rate=250, volume=.3)


def get_voice_settings(engine):
    volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
    rate = engine.getProperty('rate')   # getting details of current speaking rate
    voices = engine.getProperty('voice') #getting details of the current voice
    print (volume, rate, voices)  
    #get_voice_settings(engine)
    

def speak(text):
    engine.say(text) 
    engine.runAndWait() #Without this command, speech will not be audible to us.
    #speak("Hello World! Hope you all are doing well.")    


#init voice
engine = set_voice(gender=0, rate=185, volume=.4)


In [4]:
#VOICE GTTS

def speak_gtts(text):
    tts = gTTS(text=text, lang='en',slow=False)
    filename ='sphrase_tmp.mp3'
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)
    #speak_gtts("hello ")


def speak_gtts_language(text="hello", lang="en"):
    tts = gTTS(text=text, lang=lang,slow=False)
    filename ='phrase_tmp.mp3'
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)
    #speak_gtts_language(text="Good day I am Rocky", lang='it')

In [9]:
#Speech Recognition
#import speech_recognition as sr

#https://cloud.google.com/speech-to-text/docs/languages
#lang_dk = 'da-DK'
#lang_no = 'no-NO'
#lang_eng_GB = 'en-GB'
#lang_eng_US = 'en-US'
#lang_viet = 'vn-VN'


def takeCommand(lang='en-GB'):
    #It takes microphone input from the user and returns string output
    r = sr.Recognizer()
 
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=0.2) #testing.....can be deleted
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)
 
    
    try:
        print("Recognizing...")    
        query = r.recognize_google(audio, language=lang) #Using google for voice recognition.
        print(f"User said: {query}\n")  #User query will be printed.
    
    except Exception as e:
        print(e) # use only if you want to print the error!
        print("Say that again please...")   #Say that again will be printed in case of improper voice 
        return "None" #None string will be returned
    return query.lower()
    #query=takeCommand()
    #query=takeCommand(lang=lang_dk)
  

def dictateLanguage(query):  #dictation possible in other language
    if "danish language" in query:
        speak("Danish it is. You can dictate in danish now.")
        query_lang=takeCommand(lang=lang_dk)
    elif "norwegian language" in query:
        query_lang=takeCommand(lang=lang_no)
        speak("Norwegian it is. You can dictate in norwegian now.")
    elif "vietnamese language" in query:
        query_lang=takeCommand(lang=lang_viet)
        speak("Vietnamese it is. You can dictate in vietnamese now.")
    else:
        query_lang=query
    return query_lang
    #query = queryLanguage(query)


In [10]:
takeCommand()

Listening...
Recognizing...
User said: housing for Stornoway SIA



'housing for stornoway sia'

# functions related to assistant tasks


In [6]:
#text translation
from googletrans import Translator 
import pandas as pd

translator = Translator()
languages = googletrans.LANGUAGES
df_lang = pd.DataFrame(list(languages.items()),columns = ['lang','language']) #dict to df

def lang_abb(lang):
    lang_abb=df_lang['lang'].loc[df_lang.language==lang].values
    #print(lang_abb)
    return lang_abb


def translate_text(phrase="Hello", src="", dest="en"): 
    if len(src)<2:
        detected = translator.detect(phrase)
        src = detected.lang
    translated = translator.translate(phrase,src=src,dest=dest)     
    print(translated.text)
    print(src, dest)
    trans_text = translated.text
    return trans_text


def translate_spoken():
    speak("what do you want me to translate?")
    phrase=takeCommand()
    
    speak("To what language?")
    q_dest=takeCommand()
    dest=lang_abb(q_dest)
    
    speak("what do you want me to translate?")
    phrase=takeCommand()

    try:
        trans_text=translate_text(phrase, dest=dest[0])
        speak_gtts_language(text=trans_text, lang=dest[0])
    except:
        speak("I did not get that....here it is in korean")
        trans_text=translate_text(phrase, dest='ko')
        speak_gtts_language(text=trans_text, lang='ko')


In [7]:
#translate_to("Hej jeg hedder Rocky")
while True:
    print()
    translate_spoken()
    t=input("press return to go again")


Listening...
Recognizing...
User said: French

Listening...
Recognizing...
User said: hello I'm lucky I'm happy today

bonjour j'ai de la chance je suis heureux aujourd'hui
en fr

Listening...


KeyboardInterrupt: 

In [21]:
#Task send email
import smtplib, ssl
def serverSendEmail(to, content):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login('essalj@gmail.com', 'Primus9558g')
    server.sendmail('essalj@gmail.com', to, content)
    server.close()
    #sendEmail('essalj@gmail.com','Hejsa. Sikke en test.')


def sendEmail(query):
    #email content
    speak("What should I write?")
    q_content = takeCommand()
    content = dictateLanguage(q_content)

    #email receiver
    if "celina" in query or "selina" in query:
        to = "ccjeremiassen@gmail.com"
    elif "magnus" in query:
        to = 'mcjeremiassen@gmail.com'
    elif "to self" in query:
        to = 'essalj@gmail.com'
    else:
        speak('send to who?')
        to = takeCommand()

    try:
        serverSendEmail(to, content)
        speak("Email has been sent!")
    except Exception as e:
        print(e)
        speak("Sorry. I am not able to send this email")


In [7]:
def askWolfram():
    speak('What do you want to know?')
    question=takeCommand()

    #question = "what is the population of sweden?"
    app_id="Stones"
    client = wolframalpha.Client('4UE782-RAV5EAWPT9')
    res = client.query(question)
    answer = next(res.results).text
    speak(answer)
    print(answer)
    askWolfram()

In [8]:
def greetMe():
    hour = int(datetime.datetime.now().hour)
    if hour>=0 and hour<12:
        speak("Good Morning!")
    elif hour>=12 and hour<18:
        speak("Good Afternoon!")   
    else:
        speak("Good Evening!") 
    
#greetMe()

In [9]:
def phrase_stats(phrase):
    words = len(phrase.split())
    letters = len(phrase.replace(" ",""))
    return words, letters
#query="search google for blue dogs"
#phrase_stats(query)

# wake up call

In [10]:
def listen():
    t0 = time.time()

    #speak("How can I help you?")

    query = takeCommand() #Converting user query into lower case
    #query = "send email to lasse" #Converting user query into lower case

    #0    
    note_strs = ['note to self', 'remember this']
    for phrase in note_strs:
        if phrase in query:
            create_note(query)

    #1
    if 'wikipedia' in query:  #if wikipedia found in the query
        speak('Searching Wikipedia...')
        query = query.replace("wikipedia", "")
        results = wikipedia.summary(query, sentences=2) 
        speak("According to Wikipedia")
        print(results)
        speak(results)
        time.sleep(3)

    #2
    elif 'open youtube' in query:
        webbrowser.get('chrome').open("youtube.com")
        speak("Youtube is ready for you now")
        time.sleep(3)

    #3
    elif 'search google' in query:
        speak("what do I look for?")
        phrase = query.replace('search google',"")
        words, letters = phrase_stats(phrase)
        speak("searching google for " + phrase)

        
        #"stakeCommand()
        url = "https://www.google.com/search?query=" + phrase
        webbrowser.get('chrome').open(url)
        speak("Google is ready for you now")
        time.sleep(3)

    #4
    elif 'the time' in query:
            strTime = datetime.datetime.now().strftime("%H:%M:%S")    
            speak(f"The time is {strTime}")
            time.sleep(3)

    #5
    elif 'send email to' in query:
        sendEmail(query)
        time.sleep(3)

    #6
    elif 'the news' in query:
        url_news = "https://www.dr.dk/drtv/se/tv-avisen-morgen_229629" #dr tv-avisen play direct
        webbrowser.get('chrome').open(url_news)
        speak("The news is playing now")


    #8
    elif 'ask' in query:
        askWolfram()

   #9
    elif 'you listening' in query:
        speak("Discretely..... but yes")
        
    
     

     


In [11]:
cmd_wake_up_call = 'hello rocky'

print("")
while True:
    t0 = time.time()

    query = takeCommand()
    if query.count(cmd_wake_up_call)>0:
        speak("Rolling and ready.")
        #time.sleep(1)
        t1 = time.time()
        print(str(t1-t0))
        listen()

    elif 'you listening' in query:
        speak("Nope.....or maybe just a little")
        time.sleep(3)

    elif 'goodbye rocky' in query:
        speak("goodbye")
        time.sleep(3)
        break           
    


UnboundLocalError: local variable 'source' referenced before assignment